In [1]:
import firebase_kapp_api as f
import pandas as pd

In [205]:
csv = {
    'babymat': 'babymat.csv',
    'bakeartikler': 'bakeartikler.csv',
    'bakevarer': 'bakevarer.csv',
    'div_matprodukter': 'div_matprodukter.csv',
    'drikke': 'drikke.csv',
    'fisk': 'fisk.csv',
    'frukt_gront': 'frukt_gront.csv',
    'helsekost': 'helsekost.csv',
    'is_dessert': 'is_dessert.csv',
    'meieri_ost': 'meieri_ost.csv',
    'palegg_frokost': 'palegg_frokost.csv',
    'snacks_godteri': 'snacks_godteri.csv',
}

In [206]:
find_duplicates_all()

------ Read Data
data/babymat.csv

------ Read Data
data/bakeartikler.csv

------ Read Data
data/bakevarer.csv

------ Read Data
data/div_matprodukter.csv

------ Read Data
data/drikke.csv

------ Read Data
data/fisk.csv

------ Read Data
data/frukt_gront.csv

------ Read Data
data/helsekost.csv

------ Read Data
data/is_dessert.csv

------ Read Data
data/meieri_ost.csv

------ Read Data
data/palegg_frokost.csv

------ Read Data
data/snacks_godteri.csv

------ Find Duplicates


,category,sub_category,producer,product_type,product,kosher_type,kosher_stamp,comment
1533,pålegg & frokost,Gryn,-,Bulgur,Alle sorter uten tilsetninger er tillatte,p,-,-
1534,pålegg & frokost,Gryn,-,Couscous,Alle sorter uten tilsetninger er tillatte,p,-,-
753,fisk,"Kosher fisk, generelt i Norge",-,-,Sei,p,-,Coalfish
776,fisk,"Kosher fisk, generelt i Norge",-,-,Sei,p,-,Coalfish
1058,is & dessert,iskrem,Diplom-is,literis i butikk,Dream vanilla bean,m,-,-
1059,is & dessert,iskrem,Diplom-is,literis i butikk,Dream vanilla bean,m,-,-
1018,is & dessert,iskrem,Diplom-is,småis,Fruitero mango,p,-,-
1070,is & dessert,iskrem,Diplom-is,literis i butikk,Fruitero mango,m,-,-
1025,is & dessert,iskrem,Diplom-is,småis,Lettis vanilje,m,-,-
1026,is & dessert,iskrem,Diplom-is,småis,Lettis vanilje,m,-,-


In [207]:
# Read and load all files.
for k,v in csv.items():
    print('------------------------------------------------')
    df = read_data('data/' + v)
    insert_data(df)
    
count_products()

------------------------------------------------
------ Read Data
data/babymat.csv

------ Batch Insert Data
+ semper_spiseklar grøt med pære & sviske
+ semper_spiseklar grøt med youghert, eple og pære
+ semper_melkefri grøt med mais og banan
+ semper_havregrøt mild & naturell
+ semper_spiseklar barnegrøt med pære & aprikos
+ semper_spiseklar barnegrøt med eple & fersken
+ semper_spiseklar barnegrøt med blåbaer & eple
+ semper_spiseklar barnegrøt med pære & mango
+ semper_spiseklar barnegrøt med jordbær & banan
+ semper_fruktgrøt med pære & banan
+ semper_fullkornsgrøt med frukt og yoghurt
+ semper_fullkornsgrøt med frukt
+ semper_spiseklar mellommåltids velling
+ semper_spiseklar havrevelling
+ semper_smoothie tropisk med eple & mango
+ semper_smoothie sommer
+ semper_fruktmums fersken
+ semper_green smoothie med mango, gulrot, eple og mynte
+ semper_green smoothie med pære, avokado & lime
+ semper_green smoothie med eple, gulrot, appelsin & ingefær

--------------------------------

+ delicato_punch
+ delicato_crisp
+ delicato_nøttepunch
+ delicato_havrekokos
+ delicato_havrekokos uten sukker
+ delicato_sitronkake uten sukker
+ delicato_delicatobolle uten sukker
+ findus_kanelboller
+ findus_wienerbrød
+ findus_solskinnsboller
+ findus_pain au chocolat
+ findus_nøtteroser
+ pågen_gifler kanel
+ pågen_kanelboller
+ oreo_oreo original
+ oreo_mini
+ oreo_oreo dobbel
+ oreo_oreo crispy & thin
+ sætre_ballerina blåbærlykke
+ sætre_ballerina doble sjokokjeks
+ sætre_ballerina melkesjoko
+ sætre_ballerina original (nougat)
+ sætre_ballerina duo
+ sætre_bixit original
+ sætre_bixit sjokolade
+ sætre_bixit fullkornsbar blåbær
+ sætre_bixit turbar sjokolade
+ sætre_bixit havrebar mandler og sjokolade
+ sætre_bixit havrebar blåbær og tranebær
+ sætre_bokstavkjeks original
+ sætre_café bakeriet bare bær
+ sætre_cafe bakeriet lys sjokoladetrio
+ sætre_cafe bakeriet skogsbærdrøm
+ sætre_café bakeriet sjokoladeterapi
+ sætre_cremekjeks sitron
+ sætre_crispo melkesjokoladekj

+ santa maria_fajita spicemix
+ santa maria_vegetarian taco mix
+ santa maria_stir fry sauce original
+ mills_ekte majones
+ mills_lettmajones
+ mills_sitronmajones
+ mills_remulade
+ mills_sandwichmajones aioli
+ hellmann’s_real mayonnaise
+ vita hjertego’_lettmajones
+ info_alle vegetabilske oljer, bortsett fra druekjerneolje, er tillatte.
+ french’s_classic yellow mustard
+ idun_bodsennep
+ idun_pølsesennep
+ idun_sennep grov på glass
+ idun_sennep mild på glass
+ idun_sennep sterk på glass
+ idun_sennep uten sukker
+ slotts_senap original
+ slotts_skånsk senap, stärk och grovkornig
+ a1_steak sauce
+ heinz_tomatketchup
+ heinz_tomatpuré
+ heinz_økologisk ketchup
+ hp_hp sauce
+ kikkoman_kikkoman
+ idun_crème fraîche-dressing maks 3 % fett
+ idun_fransk dressing fettfri
+ idun_italiensk dressing fettfri
+ idun_iduns utvalgte
+ idun_hamburgerdressing
+ idun_hvitløksdressing
+ idun_hvitløksdressing maks 3 %
+ idun_ketchup red hot chilli
+ idun_partydressing
+ idun_pizzadressin

+ findus_rotgrønnsaker
+ findus_jegerblanding
+ findus_perfekt til fisk
+ findus_perfekt til kjøtt
+ findus_perfekt til lapskaus
+ findus_wok classic
+ findus_wok frisk
+ findus_wok spicy
+ findus_wok mild
+ findus_erter
+ findus_aspargesbønner
+ findus_ferskdampet økologisk
+ findus_små fine brokkolibuketter
+ findus_rosenkål
+ findus_poteto dippers
+ findus_sprø potetstaver
+ findus_criss cross
+ findus_chippy chips
+ info_alle rene grønnsaker lagt i saltvann uten eddik er kosher.
+ info_alle typer tomatpuré er kosher.
+ info_all hermetisk frukt i sin egen lake er tillatt, med mindre den inneholder druesaft.

------------------------------------------------
------ Read Data
data/helsekost.csv

------ Batch Insert Data
+ dr. schär_alle produkter er kosher
+ finax_chunky chokolate
+ finax_chunky chokolate småkaka
+ finax_crispy coconut
+ finax_glutenfri grov mjölmix utan mjölk
+ finax_glutenfri lågprotein mjölmix
+ finax_glutenfri mjölmix med mjölk
+ finax_naturligt glutenfr

+ diplom-is_diplom-is lemon'n lime sorbet
+ diplom-is_diplom-is lollipop orginal
+ diplom-is_diplom-is mangosorbet
+ diplom-is_diplom-is nugatti
+ diplom-is_diplom-is pistasj
+ diplom-is_diplom-is popcorn & salt karamell
+ diplom-is_diplom-is rom rosin
+ diplom-is_diplom-is salt karamell
+ diplom-is_diplom-is salt lakris
+ diplom-is_diplom-is sirkus
+ diplom-is_diplom-is sitron
+ diplom-is_diplom-is sjokolade
+ diplom-is_diplom-is solbærsplitt
+ diplom-is_fri vanilje
+ diplom-is_laktosefri vaniljeis
+ diplom-is_milkshake
+ diplom-is_softis med vaniljesmak
+ hennig-olsen_crème premium karamell
+ hennig-olsen_dyreparken
+ hennig-olsen_båt is
+ hennig-olsen_big deal
+ hennig-olsen_crème cappuccino
+ hennig-olsen_crème mandel
+ hennig-olsen_crème melkesjokolade
+ hennig-olsen_crème pasjonsfrukt & sitron
+ hennig-olsen_crème premium nougat
+ hennig-olsen_daim balltop
+ hennig-olsen_dugg sorbet eple & appelsin
+ hennig-olsen_dugg yoghurtis bringebær & jordbær
+ hennig-olsen_friskis ap

+ tine_biola uten tilsatt sukker granateple bringebær
+ tine_biola laktosefri fettfri syrnet melk blåbær
+ tine_matyoghurt
+ tine_matyoghurt tzatziki
+ tine_yoghurt blåbær
+ tine_yoghurt flid kirsebær
+ tine_yoghurt flid solbær
+ tine_yoghurt jordbær
+ tine_yoghurt kirsebær
+ tine_yoghurt melon
+ tine_yoghurt molte
+ tine_yoghurt skogsbær
+ tine_yoghurt appelsin
+ tine_yoghurt mango pasjonsfrukt
+ tine_yoghurt kokos
+ tine_yoghurt nyt blåbær og bringebær
+ tine_yoghurt nyt eple kanel og pære
+ tine_yoghurt sommerfrisk, alle typer
+ tine_yoghurt sesong høst vinter
+ tine_yoghurt vårfrisk, alle typer
+ tine_yoghurt sviske
+ tine_yoghurt vanilje
+ tine_piano® duo yoghurt sjokolade og sjokocrisp
+ tine_piano® duo yoghurt nøttetoffee og sjokocrisp
+ tine_go'morgen jordbæryoghurt og müsli
+ tine_go'morgen mango bananyoghurt og müsli
+ tine_go'morgen melon pasjonsfruktyoghurt med 4-kornmüsli
+ tine_go'morgen skogsbæryoghurt og müsli
+ tine_go'morgen vaniljeyoghurt med nøttemix
+ tine_

Database: 1798



In [204]:
# Read and load a single file.
df = read_data('data/' + csv['snacks_godteri'])
find_duplicates(df)
# insert_data(df)
count_products()

------ Read Data
data/snacks_godteri.csv

------ Find Duplicates


,category,sub_category,producer,product_type,product,kosher_type,kosher_stamp,comment



------ Count Products
babymat.csv: 20
bakeartikler.csv: 129
bakevarer.csv: 258
div_matprodukter.csv: 209
drikke.csv: 128
fisk.csv: 104
frukt_gront.csv: 23
helsekost.csv: 105
is_dessert.csv: 314
meieri_ost.csv: 193
palegg_frokost.csv: 58
snacks_godteri.csv: 281
---
All files: 1822
Database: 281



In [112]:
def read_data(csv_file):
    """
    Also presents some information about the data.
    Returns: A Panda DataFrame.
    """
    print("------ Read Data")
    print(csv_file)
    df = pd.read_csv(csv_file)
    print()
    return df

In [155]:
def find_duplicates(df):
    """
    Find duplicates in the dataframe.
    Duplicates is checked on producer+product.
    """
    print('------ Find Duplicates')
    dups = df.duplicated(subset=['producer','product'], keep=False) 
    df_dups = df[dups].sort_values(by=['producer','product'])
    display(df_dups)
    print()

In [157]:
def find_duplicates_all():
    """
    """
    df = pd.DataFrame()
    for k,v in csv.items():
        df = pd.concat([df, read_data('data/' + v)], ignore_index=True)
    find_duplicates(df)

In [111]:
def insert_data(df):
    """
    Creates products in the Firebase database.
    Input: A DataFrame
    """
    print("------ Batch Insert Data")
    f.batch_create(df)
    print()

In [116]:
def count_products():
    """
    Count products in CSV files and in the database.
    """
    print("------ Count Products")
    csv_len_sum = 0
    for k,v in csv.items():
        df_len = len(pd.read_csv('data/' + v))
        csv_len_sum = csv_len_sum + df_len
        print("{}: {}".format(v, df_len))
    print("---")
    print("All files: {}".format(csv_len_sum))
    print("Database: {}".format(f.nofProducts()))
    print()

In [165]:
f.list_products(False)

id:-_

product:


id:-_akevitt
product:Akevitt

id:-_amaretto
product:Amaretto

id:-_arak pastis
product:Arak Pastis

id:-_bacardi
product:Bacardi

id:-_bailey’s original irish cream
product:Bailey’s Original Irish Cream

id:-_bols liqueur
product:Bols Liqueur

id:-_calvados
product:Calvados

id:-_campari bitter
product:Campari Bitter

id:-_cherry brandy
product:Cherry Brandy

id:-_cider
product:Cider

id:-_cointreau
product:Cointreau

id:-_drambuie
product:Drambuie

id:-_eggelikør
product:Eggelikør

id:-_gamla cabernet sauvignon
product:Gamla Cabernet Sauvignon

id:-_gamla nature brut
product:Gamla Nature Brut

id:-_gamla sangiovese
product:Gamla Sangiovese

id:-_gin
product:Gin

id:-_irish cream
product:Irish Cream

id:-_kahlua
product:Kahlua

id:-_kirschwasser
product:Kirschwasser

id:-_malibu
product:Malibu

id:-_mozartlikør
product:Mozartlikør

id:-_ouzo
product:Ouzo

id:-_pernod
product:Pernod

id:-_peter heering
product:Peter Heering

id:-_rom
product:Rom

id:-_sambuca
product:S

id:toro_kålstuing
product:Kålstuing

id:toro_kjøttdeigsaus
product:Kjøttdeigsaus

id:toro_kremet spinatsuppe
product:Kremet spinatsuppe

id:toro_kruspersille
product:Kruspersille

id:toro_krydder bearnaisesaus
product:Krydder Bearnaisesaus

id:toro_kyllingkrydder
product:Kyllingkrydder

id:toro_lammekjøttkrydder
product:Lammekjøttkrydder

id:toro_laurbærblad
product:Laurbærblad

id:toro_lett bearnaisesaus
product:Lett Bearnaisesaus

id:toro_løkpulver
product:Løkpulver

id:toro_majoran
product:Majoran

id:toro_malt allehånde
product:Malt allehånde

id:toro_malt cayennepepper spansk pepper
product:Malt Cayennepepper Spansk Pepper

id:toro_malt ingefær
product:Malt ingefær

id:toro_malt kardemomme
product:Malt kardemomme

id:toro_malt koriander
product:Malt koriander

id:toro_malt muskatblomme
product:Malt muskatblomme

id:toro_malt muskatnøtt
product:Malt muskatnøtt

id:toro_malt nellik
product:Malt nellik

id:toro_malt stjerneanis
product:Malt stjerneanis

id:toro_mexicansk chili-pu

In [167]:
# %%time
def get_docs():
    docs = f.products_ref().get()
    products = []
    for doc in docs:
        products.append(doc.to_dict())
    return products

p = get_docs()
p

[{'producer': '-',
  'kosher_type': 'p',
  'product_type': '-',
  'sub_category': 'Alkohol, Vin',
  'kosher_stamp': '-',
  'product': '\n',
  'category': 'drikke',
  'ts_created': DatetimeWithNanoseconds(2018, 12, 29, 19, 21, 56, 700000, tzinfo=<UTC>),
  'comment': '-'},
 {'product': 'Akevitt',
  'category': 'drikke',
  'ts_created': DatetimeWithNanoseconds(2018, 12, 29, 19, 21, 56, 912000, tzinfo=<UTC>),
  'comment': '-',
  'producer': '-',
  'kosher_type': 'p',
  'product_type': '-',
  'sub_category': 'Alkohol, Vin',
  'kosher_stamp': '-'},
 {'ts_created': DatetimeWithNanoseconds(2018, 12, 29, 19, 21, 57, 127000, tzinfo=<UTC>),
  'comment': '-',
  'producer': '-',
  'kosher_type': 'p',
  'product_type': '-',
  'sub_category': 'Alkohol, Vin',
  'kosher_stamp': '-',
  'product': 'Amaretto',
  'category': 'drikke'},
 {'producer': '-',
  'kosher_type': 'p',
  'product_type': '-',
  'sub_category': 'Alkohol, Vin',
  'kosher_stamp': '-',
  'product': 'Arak Pastis',
  'category': 'drikke',


In [15]:
docs = f.products_ref().get()

type(docs)

generator

### Misc

In [3]:
f.create({
    u'category': u'REMOVE',
    u'comment': u'',
    u'kosher_stamp': u'',
    u'kosher_type': u'p',
    u'producer': u'Toro',
    u'product': u'suppe',
    u'sub_category': u'',
})

x toro_suppe already exists


In [173]:
doc_ref1 = f.get_doc_ref(u'-_\n')

f.pp(doc_ref1)

Product (-_
) does not exists


In [9]:
# doc_ref1 = f.get_doc_ref(u'J7Rh7IcdEZ3Sxnb0N1yO')
# doc_ref2 = f.delete(f.get_doc_ref(u'J7Rh7IcdEZ3Sxnb0N1yO'))
doc_ref3 = f.get_doc_ref(u'213132')

f.update(doc_ref3, {u'comment':'Bye'})

Product (213132) does not exists
Product (213132) does not exists


In [10]:
doc_ref1 = f.get_doc_ref(u'J7Rh7IcdEZ3Sxnb0N1yO')
doc_ref2 = f.delete(f.get_doc_ref(u'nTk2VlJoA9MOXxZuMrBs'))

f.pp(doc_ref1)
display(f.isDeleted(doc_ref1))
display(f.exists(doc_ref1))

Product (J7Rh7IcdEZ3Sxnb0N1yO) does not exists
Product (nTk2VlJoA9MOXxZuMrBs) does not exists
Product (nTk2VlJoA9MOXxZuMrBs) does not exists


False

False

In [7]:
s = "Håndværker m/solsikkekjerner"
s
s.encode('unicode-escape')

'Håndværker m/solsikkekjerner'

b'Ha\\u030andv\\xe6rker m/solsikkekjerner'